In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from mysql.connector import connect
from typing import List
import time, random
from urllib.parse import urljoin
import os

Retrieving 10 pages data of machanical keyboard from Amazon using BeautifulSoup Library

In [2]:
def get_body(url):
    print("Connecting...")
    header = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Connection": "keep-alive",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15"

            }
    response = requests.get(url,headers=header)
    if response.status_code == 200:
        html_body = response.content
        soup = BeautifulSoup(html_body,'html.parser')
    else:
        soup = f"Fail! Could not connect with url: {response.status_code}"
    return soup

def get_links(url):
    soup = get_body(url)
    list_link = []
    links = soup.find_all("a",attrs={"class":"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"})
    list_link = [link["href"] for link in links]
    return list_link

def get_data(links:List) -> pd.DataFrame:
    all_product_data =[]

    #exact
    base_url = "https://www.amazon.com"
    for link in links:
 
        #initilize data dict 
        product_data = {
        "Title": None,
        "Link":None,
        "Brand": None,
        "Price":None,
        "Review":None,
        "Total Review":None,
        "Compatible Devices": None,
        "Color": None,
        "Number of Keys": None,
        "Connectivity Technology": None,
        "Rank":None
                    }
        time.sleep(random.uniform(1, 3))
        new_url = urljoin(base_url, link)
        soup = get_body(new_url)

        product_data["Link"] =  new_url

        #product title
        h1 = soup.find("span",attrs={"id":"productTitle"})
        try:
            h1 = soup.find("span", attrs={"id": "productTitle"})
            product_data["Title"] = h1.text.strip() if h1 else None
        except AttributeError:
            product_data["Title"] = None
        else:
            print("Complete title part")
            
        ##review
        review_container = soup.find("div",attrs={"id":"customerReviews"})
        try:
            review = review_container.find("span",attrs={"data-hook":"rating-out-of-text"})
            product_data["Review"] = review.text
        except AttributeError:
            product_data["Review"] = None
        else:
            print("Complete review part")


        #total review
        try:
            review_total = review_container.find('span',attrs={"data-hook":"total-review-count"}).text
            product_data["Total Review"] = review_total
        except AttributeError:
            product_data["Total Review"] = None
        else:
            print('Complete total review part')

        ##price
        try:
            price_symbol = soup.find(class_='a-offscreen').text
            product_data["Price"] = price_symbol
        except AttributeError:
            product_data["Price"] = None
        else:
            print("Complete price part")

        # print(product_detail)
        product_detail = soup.find(class_="a-normal a-spacing-micro")
        try:
            for row in product_detail.find_all('tr'):
                name = row.find('td',attrs={"class":"a-span3"}).text.strip()
                feature = row.find('td',attrs={"class":"a-span9"}).text
                if name in product_data:
                    product_data[name] = feature
        except AttributeError:
            product_data[name] = None
        else:
            print("Complete")
            # print(name.text, "-", feature.text)

        #rank
        addition_value = soup.find("table",attrs={"id":"productDetails_detailBullets_sections1"})
        try:
            for row in addition_value.find_all('tr'):
                header = row.find('th').text
                if "Best Sellers Rank" in header:
                    rank = row.find("td").text
                    product_data["Rank"] = rank
        except AttributeError:
            product_data["Rank"] = None
        else:
            print("Complete rank part")
        
        print("Finish link %s"%link)
        
        all_product_data.append(product_data)
    else:
        data = pd.DataFrame(all_product_data)
        return data
    
def retrieve_page_data(start,end=None):
    df=[]
    if end is None:
        end = start
        start = 1
    
    for i in range(start,end):
        url=f"https://www.amazon.com/s?k=Computer+Keyboards&i=computers&rh=n%3A12879431&page=2&c=ts&qid=1724479075&ts_id=12879431&ref=sr_pg_{i}"
        list_links = get_links(url)
        data = get_data(list_links)
        df.append(data)
    combine = pd.concat(df,ignore_index=True)

    print(f"Finshed retrieving data from page {start} to {end-1}")
    return combine

def combine_all_df(*args) -> pd.DataFrame:
    df = pd.concat(args,ignore_index=True)

    print(f"Finish combining {len(args)} DataFrames")
    return df

def write_csv(foldername,filename,data:pd.DataFrame):
    present_path = os.getcwd()
    folder_path = os.path.join(present_path,foldername)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {foldername} is created in {present_path}")
    
    file_path = os.path.join(folder_path,filename)
    data.to_csv(file_path, index=True)
    print(f"Data saved to {file_path}")


In [3]:
#fetch data of first 2 pages
data_part1 = retrieve_page_data(3)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
Finish link /sspa/click?ie=UTF8&spc=MTo1NjM1OTEzNTg0MzU4MTM6MTcyNDU4NjA0MTpzcF9hdGZfbmV4dDozMDAwODAwNjExMzY3MDI6OjA6Og&url=%2FKeyboard-Computer-Multimedia-Spill-Resistant-Anti-Wear%2Fdp%2FB0C9GKH53F%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.WLPYv7olHfGTKnbUwUSLEQ2hjPPs2IrGkAueCxYvmv0M_q_9GAzPSulqjnvtBU8pDVw06w1Ff2T2MHVFkBPqTsQht_z66KjVtGPhcM53IeHY9jNnC-5VrmgOKtsghzO4vWKKYnL3ZJHf8Y7rrHB3ZSmWD0_whjwtyWsl4ErbGuwJwfvihvFCgS_BkILfvUN8niVOkIPRCpmutDW5nw-jvmKn9xLjRIIAw53OWfSH9ryFCnpV4Qr9PGw_Ea93tYIxUkTtM0E6aqf-fLqQlwQVjQrGSNboPS9dpfcEy95NiyI.WQLDltdVkOiQptxNMIRDDqFFyAeXcHM2-SgUVWd89HI%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724586040%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price

In [4]:
data_part1.shape

(62, 11)

In [5]:
data_part2=retrieve_page_data(3,5)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
Finish link /sspa/click?ie=UTF8&spc=MTo1Mjg2MDY0MzY4NjU3MzUwOjE3MjQ1ODY0MjM6c3BfYXRmX25leHQ6MzAwMDgwMDYxMTM2NzAyOjowOjo&url=%2FKeyboard-Computer-Multimedia-Spill-Resistant-Anti-Wear%2Fdp%2FB0C9GKH53F%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.IyPxMdATxtNcyyhLMlyeZ8IKPUNtPuJUCGG7MpyI9Ss3AcYaNLCHnyhkXiTwXKauSr4wHrReNQj3zzh19DazqmmOm61KSoHX6BdO_MdNoCAw3oXMWg6IUQY7EsfkE1GIWBGj9PBPfSqIUrJ8fAoQMCg-4WcVRYAQSF87gY9Oa5if2HkcJgnKjUZzqWPAeax1Dzs4JGrp7jsSl_ZxsnCgu4S0SVBUdQWQX6bWR5Lp7IoroSs7FdwRUfxly5JzaYfDCVb0WqI74OFx4Eo8cqF3Gs-Pi3M02LJ7M_xt3aMa2zA.BCgNY8UBSEnBwEjcQU5dBpHd-V6m6z8CggduJjOBW4w%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724586423%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete pric

In [6]:
data_part2.shape

(62, 11)

In [7]:
data_part3 = retrieve_page_data(5,7)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
Finish link /sspa/click?ie=UTF8&spc=MToxMDc1MTMwODg5NjQzNTI1OjE3MjQ1ODY4OTY6c3BfYXRmX25leHQ6MzAwMDgwMDYxMTM2NzAyOjowOjo&url=%2FKeyboard-Computer-Multimedia-Spill-Resistant-Anti-Wear%2Fdp%2FB0C9GKH53F%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.x4TW_9SqknpnqHKz73mE6dxxp9VwJJvBYV4hpGRwRHgPuBfBGDO2De4AYSeAy7cNGQ3S18epl-9zs_MMhUQMp2kIHCuV6xtK5ycgT_E4tCz5lCFcw0yBOg9GjQqTS91FWPQ1b38rf8cgYp87FTl75uINDkhSlX49rHpO18_6yDal3rm_SNiEPoscdsMPhV-3m7kUuRQyqYrKMuHaJpvXEK6rMY2Bwdj9YbZJKt54jpj_FNfR5lPalW7M6shfcwTHAqRGF-oEh-9Y15YhxmnhdZTFkBnIukNH2-0fhmzhaW8.CTw7EW53I72Zls6VMqZw9C6afGevbOspoW6Cd8_CCXI%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724586896%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete pric

In [8]:
data_part3.shape

(63, 11)

In [9]:
data_part4 = retrieve_page_data(7,9)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
Finish link /sspa/click?ie=UTF8&spc=MTo0Njk4ODM4NjgzMTQxNzEyOjE3MjQ1ODczNTI6c3BfYXRmX25leHQ6MzAwMDgwMDYxMTM2NzAyOjowOjo&url=%2FKeyboard-Computer-Multimedia-Spill-Resistant-Anti-Wear%2Fdp%2FB0C9GKH53F%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.IyPxMdATxtNcyyhLMlyeZ8IKPUNtPuJUCGG7MpyI9Ss3AcYaNLCHnyhkXiTwXKauobsgT1I8EDOYzuX_fWJgfejUItPdMVXL-z3JCY17rKI2bT8d82I1L5O4PCFsRpEleGeXFoZ0s3P5y6ZYioqTTSg-4WcVRYAQSF87gY9Oa5if2HkcJgnKjUZzqWPAeax1Dzs4JGrp7jsSl_ZxsnCgu4S0SVBUdQWQX6bWR5Lp7IoroSs7FdwRUfxly5JzaYfDwr7qOekKfGEK_0VCAhhQz31XxMVEGEIKRYfa5hH31p4.hOjN4bzsI3cgrDkXPX7XmYTOpijc8pdvuuBDxbTruMs%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724587352%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete pric

In [10]:
data_part4.shape

(62, 11)

In [11]:
data_part5 = retrieve_page_data(9,11)

Connecting...
Connecting...
Complete title part
Complete review part
Complete total review part
Complete price part
Complete
Complete rank part
Finish link /sspa/click?ie=UTF8&spc=MTo0OTAyNDE3NzY1MjY4MzAwOjE3MjQ1ODc3OTU6c3BfYXRmX25leHQ6MzAwMDgwMDYxMTM2NzAyOjowOjo&url=%2FKeyboard-Computer-Multimedia-Spill-Resistant-Anti-Wear%2Fdp%2FB0C9GKH53F%2Fref%3Dsr_1_25_sspa%3Fc%3Dts%26dib%3DeyJ2IjoiMSJ9.IyPxMdATxtNcyyhLMlyeZ8IKPUNtPuJUCGG7MpyI9Ss3AcYaNLCHnyhkXiTwXKauSr4wHrReNQj3zzh19DazqmmOm61KSoHX6BdO_MdNoCBYW6qaMVVeSX_gfnSPa6GFkuoekmRthgv-spC31Xo2aZ0Ud4IuoMHND-RtAS85uYmf2HkcJgnKjUZzqWPAeax1Dzs4JGrp7jsSl_ZxsnCgu4S0SVBUdQWQX6bWR5Lp7IoroSs7FdwRUfxly5JzaYfDwr7qOekKfGEK_0VCAhhQz31XxMVEGEIKRYfa5hH31p4.81uNaaVLyArub7rQHH-1CQZyBq3vUO4aYp0EGt1jz8w%26dib_tag%3Dse%26keywords%3DComputer%2BKeyboards%26qid%3D1724587795%26s%3Dpc%26sr%3D1-25-spons%26ts_id%3D12879431%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1
Connecting...
Complete title part
Complete review part
Complete total review part
Complete pric

In [12]:
data_part5.shape

(62, 11)

In [13]:
data = combine_all_df(data_part1,data_part2,data_part3,data_part4,data_part5)
write_csv("Data","Products",data)

Finish combining 5 DataFrames
Folder Data is created in /Users/huyenhoang/Documents/Data-Projetcs/Amazon Project/Crawl data
Data saved to /Users/huyenhoang/Documents/Data-Projetcs/Amazon Project/Crawl data/Data/Products


Load data to database

In [14]:
# create functions for interaction with database
def connect_to_mysql(host,user,password,port,database = None):
    try:
        if database:
            conn= connect(host=host, user=user, password=password,port=port,database=database)
        else:
            conn = connect(host=host, user=user, password=password,port=port)
        return conn
    except Exception as e:
        print(f"Error: {e}")

def create_database(cursor,db):
    try:
        cursor.execute(f"CREATE DATABASE {db}")
        print(f"Create successfully database {db}")
    except Exception as e:
        print(f"Error: {e}")

def create_table(cursor,table_name):
    sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        link TEXT,
        brand VARCHAR(255),
        price VARCHAR(255),
        review VARCHAR(255),
        total_review VARCHAR(255),
        compatible_devices TEXT,
        color VARCHAR(255),
        numkeys INT,
        connect_tech VARCHAR(255),
        rank_amz VARCHAR(255)
        )
    """

    try:
        cursor.execute(sql)
        print(f"Table {table_name} is created successfully")
    except Exception as e:
        print(f"Error: {e}")

def transfer_data(cursor,df, table_name):
    insert_sql = f"""
        INSERT INTO {table_name}(title, link, brand, price, review, total_review, compatible_devices, color, numkeys, connect_tech, rank_amz) 
        VALUE(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)     
        """
    data_to_insert = df.values.tolist()
    try:
        cursor.executemany(insert_sql,data_to_insert)
        print(f"Data inserted into table {table_name}")
    except Exception as e:
        print(f"Error: {e}")


In [15]:
host = "host"
user = "user"
password = "password"
port = "port"

#create connection
conn = connect_to_mysql(host=host,user=user,password=password,port=port)
conn.is_connected()

True

In [16]:
cursor = conn.cursor()
create_database(cursor,"amazon")

Error: 1007 (HY000): Can't create database 'amazon'; database exists


In [17]:
conn.database = "amazon"
create_table(cursor,"keyboards")
conn.commit()

Table keyboards is created successfully


In [18]:
#transfer data
try:
    transfer_data(cursor,data,"keyboards")
    conn.commit()
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

Data inserted into table keyboards
